In [182]:
import pandas as pd
import numpy as np
import os
# import seaborn as sns
# import plotly.express as px
# import plotly.graph_objs as go
# from plotly.tools import FigureFactory as FF
import re
import os
from datetime import datetime, timedelta 

df = pd.DataFrame()
os.getcwd()
 

'c:\\Users\\dscshap3808\\Documents\\my_scripts_new\\play4_activity'

In [682]:
# pd.read_excel('./data/Coach11-1to11-5.xlsx', sheet_name=0)\
#     .to_csv('./data/coach_15.csv', encoding = 'utf_8_sig', index=None)
coach = pd.read_csv('./data/coach_15.csv')
coach = coach.dropna(axis=1, how = 'all')
# task_collect = coach.groupby(['姓名', '开始日期'])['任务类型'].apply(list).astype(str).reset_index();task_collect
# onoff = task_collect[task_collect['任务类型'].str.contains('(下班.+上班)')][['姓名','开始日期']].sum(axis = 1)
# coach['flag'] = coach[['姓名','开始日期']].sum(axis = 1)
coach['开始时间'] = pd.to_datetime(coach['开始时间'])
coach['结束时间'] = pd.to_datetime(coach['结束时间'])

# coach_nopunch = coach[~coach['flag'].isin(onoff)]
# coach_dopunch = coach[ coach['flag'].isin(onoff)]
coach['sprm'] = (60/coach['SPR标准数量']).replace([np.inf, -np.inf], 0)

wh = coach.groupby(['姓名','开始日期']).agg(
    {
        '开始时间': 'min',
        '结束时间': 'max',
        'sprm': 'sum',
    }
).reset_index()
wh['work_hour'] = wh['结束时间'] - wh['开始时间']
wh['work_hour_in_min']  = [i.total_seconds()/60 for i in wh['work_hour']]
wh['work_hour_in_hour'] = [i.total_seconds()/3600 for i in wh['work_hour']]
wh = wh.drop(['结束时间','开始时间', 'work_hour'], axis = 1).rename({'sprm':'SPRM_total_of_day'}, axis =1)
coach = coach.merge(wh, on = ['姓名', '开始日期'], how = 'inner')
coach['sprm_perhour'] =  coach['SPRM_total_of_day'] / coach['work_hour_in_hour']

# 标准时长计算
coach = pd.concat([coach, pd.DataFrame(list(coach['调整后持续时间'].str.split(':')))], axis =1).drop('序号',axis =1)
coach = coach[coach[[0,1,2]].astype(int).sum(axis = 1) != 0]
coach[[0,1,2]] = coach[[0,1,2]].astype(int)
coach['time_len'] = coach[0]*3600 + coach[1]*60 + coach[2]
coach = coach.drop([0,1,2], axis = 1)

In [446]:
# coach[coach['姓名'] == '孙清元']

In [402]:

import plotly.express as px
import plotly.graph_objects as go

In [428]:
# plot_data('2021-11-05').info()
# coach['开始日期'].unique()
# array(['2021-11-05', '2021-11-04', '2021-11-03', '2021-11-02',
#        '2021-11-01'], dtype=object)

In [685]:
def plot_data(riqi):
    data = coach[coach['开始日期'] == riqi][['姓名', '工作组', '直属上级', '任职时间', '职位',\
    'sprm', 'SPRM_total_of_day', 'work_hour_in_min',
    'work_hour_in_hour', 'sprm_perhour']].drop_duplicates()
    data['工作组'] = data['工作组'].fillna('unknow')
    data['工作组'] = data['工作组'].fillna('unknow')

    return data
def plot_(riqi):
    fig2 = px.scatter(
        data_frame = plot_data(riqi), x = '姓名', y= 'sprm_perhour', hover_data=['工作组' ],
        size = 'work_hour_in_hour', color = '工作组', 
        # facet_col = '开始日期', 
        title= "报告日期" + riqi)  
    # hover_data=  ['account_name_en'] , 


    # fig2.add_hline(y = line_20, line_width=3, line_dash="dash", line_color="black", opacity = .4,\
    #     annotation_text="2020 <br> weighted mean profit_rate <br>, %s" %line_20 , 
    #     annotation_position="top right", annotation_opacity = .4, col = 1)
    # fig2.add_hline(y = line_21, line_width=3, line_dash="dash", line_color="black", opacity = .4,\
    #     annotation_text="2021 <br> weighted mean profit_rate <br>, %s" %line_21 , 
    #     annotation_position="top right", annotation_opacity = .4, col = 2)

    return fig2

plot_('2021-11-03')

In [283]:
# SELECT  * FROM dm_dsc_smart.dwd_task
# where substr(station_name, 1,4) = 'COAC'
# and work_group_id != 0 and work_group_name != ''
# and inc_day = '20211127'
 

In [534]:
# coach = coach.dropna(how = 'all', axis = 1)
    # time_convert()
def time_convert(col):
    coach[col] = coach[col].astype(int)
    coach[col] = [datetime.fromtimestamp(i).strftime('%Y-%m-%d %H:%M:%S') for i in coach[col]]
    return coach
for i in ['start_time', 'end_time', 'hire_time', 'update_time']:
    time_convert(i)
coach['start_time'] = pd.to_datetime(coach['start_time'])
coach['end_time']   = pd.to_datetime(coach['end_time'])
coach['sprm'] = (60/coach['work_content_refer']).replace([np.inf, -np.inf], 0)


# wh = coach.groupby(['worker_name','start_date']).agg(
#         {
#             'start_time': 'min',
#             'end_time': 'max',
#             'sprm': 'sum',
#         }
#     ).reset_index()
# wh['work_hour'] = wh['结束时间'] - wh['开始时间']
# wh['work_hour_in_min']  = [i.total_seconds()/60 for i in wh['work_hour']]
# wh['work_hour_in_hour'] = [i.total_seconds()/3600 for i in wh['work_hour']]
# wh = wh.drop(['结束时间','开始时间', 'work_hour'], axis = 1).rename({'sprm':'SPRM_total_of_day'}, axis =1)
# coach = coach.merge(wh, on = ['姓名', '开始日期'], how = 'inner')

In [541]:
coach[coach[['duration',
    'adjustment_duration', 'adjusted_duration', ]].sum(axis = 1) != 0]

,station_name,worker_name,work_group_name,up_worker_name,hire_time,worker_post_name,worker_level_name,work_num,start_time,end_time,...,min_operate_volume,work_content,work_content_refer,work_content_is_measure,start_date,end_date,report_date,create_time,update_time,sprm
4052,COACH SHA WGQ WHS,孙孟孟,NaN,Michelle Wu,1970-01-01 08:00:00,操作员,操作员,0.0,2021-11-27 06:25:02,2021-11-27 06:25:13,...,0.0,叉车安全检查,0,0,2021-11-27,2021-11-27,2021-11-27,1638231191,2021-11-30 08:13:11,0.0
4053,COACH SHA WGQ WHS,孙孟孟,NaN,Michelle Wu,1970-01-01 08:00:00,操作员,操作员,0.0,2021-11-27 06:25:13,2021-11-27 15:25:02,...,0.0,无效时间,0,0,2021-11-27,2021-11-27,2021-11-27,1638231191,2021-11-30 08:13:11,0.0


In [511]:
# # coach= coach.drop(['dwd_task_id', 'station_id', 'work_turn_id', 'raw_data'], axis  =1)

# coach['work_content'].value_counts()
# coach['work_content_refer'].value_counts()
# coach

In [477]:
# %%time
# df = pd.DataFrame()
# for i in np.arange(coach.shape[0]):
#     df = pd.concat([pd.DataFrame.from_dict(json.loads(coach['raw_data'][i]), orient="index" ).T, df], axis = 0)

In [689]:
coach = pd.read_csv('./data/coach_bdp_all.csv', sep = '\001')
coach = coach.dropna(how = 'all', axis = 1)
coach.columns = [re.sub('\w+\.', '', i) for i in list(coach.columns)]
coach = coach[['station_name','worker_name', 'work_group_name', 'up_worker_name',\
    'hire_time', 'worker_post_name', 'work_num', 'start_time', 'end_time', 'duration',
    'adjustment_duration', 'adjusted_duration', 'sku_no', 'min_package_quantity', 'min_operate_length', 'min_operate_width',
    'min_operate_height', 'min_operate_weight', 'min_operate_volume',
    'work_content', 'work_content_refer', 'work_content_is_measure',
    'start_date', 'end_date', 'report_date', 'create_time', 'update_time']]
print(coach.shape)

C:\Users\dscshap3808\Miniconda3\envs\siming\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning:

Columns (9,37,42) have mixed types.Specify dtype option on import or set low_memory=False.



(57315, 27)


In [653]:
# coach[coach['start_date'] == '2021-11-23'] 

In [663]:
def plot0_date(date, col):
    # sns.histplot(
    return coach[coach['start_date'] == date][col].value_counts()

In [673]:
plot0_date('2021-11-28', 'work_content_refer')
# pd.set_option("display.max_rows", 15, "display.max_columns", None)
# coach.head()

0      5614
360     786
15      143
Name: work_content_refer, dtype: int64

In [690]:
def data_prepare(coach):

    """
    time unix convert, 
    转换后标准时长换算成秒,
    sprm 计算.
    工作在勤时间
    """
    coach = coach.dropna(how = 'all', axis = 1)
    
    # time_convert()
    def time_convert(col):
        coach[col] = coach[col].astype(int)
        coach[col] = [datetime.fromtimestamp(i).strftime('%Y-%m-%d %H:%M:%S') for i in coach[col]]
        return coach
    for i in ['start_time', 'end_time', 'hire_time', 'update_time']:
        time_convert(i)
    coach['start_time'] = pd.to_datetime(coach['start_time'])
    coach['end_time']   = pd.to_datetime(coach['end_time'])
    # coach['duration'] =  coach['end_time'] - coach['start_time']
    # coach = coach[coach['duration'] != '0']
    # sprm calculation
    coach['sprm'] = (60/coach['work_content_refer']).replace([np.inf, -np.inf], 0)
    wh = coach.groupby(['worker_name','start_date']).agg(
        {
            'start_time': 'min',
            'end_time': 'max',
            'sprm': 'sum',
        }
    ).reset_index()
    wh['work_hour'] = wh['end_time'] - wh['start_time']
    wh['work_hour_in_min']  = [i.total_seconds()/60 for i in wh['work_hour']]
    wh['work_hour_in_hour'] = [i.total_seconds()/3600 for i in wh['work_hour']]
    wh = wh.drop(['end_time','start_time', 'work_hour'], axis = 1).rename({'sprm':'SPRM_total_of_day'}, axis =1)
    coach = coach.merge(wh, on = ['worker_name', 'start_date'], how = 'inner')
    # coach = coach[coach['duration'] != '0']
    coach = coach[coach['sprm'] != 0]
    coach['sprm_perhour'] =  coach['SPRM_total_of_day'] / coach['work_hour_in_hour']
    """
    计算转换后时间长度, 换算成 
    秒
    """
    # coach = pd.concat([coach, pd.DataFrame(list(coach['adjusted_duration'].str.split(':')))], axis =1) 
    # coach = coach[coach[[0,1,2]].astype(int).sum(axis = 1) != 0]
    # coach[[0,1,2]] = coach[[0,1,2]].astype(int)
    # coach['time_len'] = coach[0]*3600 + coach[1]*60 + coach[2]
    # coach = coach.drop([0,1,2], axis = 1) 
    
    return coach
coach = data_prepare(coach)


In [692]:
# coach[~coach['work_group_name'].isna()]
coach['start_date'].astype(str).str.replace('-', '')

31921    20211128
31922    20211128
31923    20211128
31924    20211128
31925    20211128
           ...   
44339    20211129
44340    20211129
44341    20211129
44342    20211129
45228    20211129
Name: start_date, Length: 2490, dtype: object

In [681]:
def plot_data(riqi):
    data = coach[coach['start_date'] == riqi][['worker_name', 'work_group_name', 'up_worker_name', 'hire_time', 'worker_post_name',\
    'sprm', 'SPRM_total_of_day', 'work_hour_in_min',
    'work_hour_in_hour', 'sprm_perhour']].drop_duplicates()
    data['work_group_name'] = data['work_group_name'].fillna('unknown')
    return data
def plot_(riqi):
    fig2 = px.scatter(
        data_frame = plot_data(riqi), x = 'worker_name', y= 'sprm_perhour',
        size = 'work_hour_in_hour', color = 'work_group_name',  hover_data=['work_group_name'],
        # facet_col = '开始日期', 
        title= "报告日期" + riqi)  
    # hover_data=  ['account_name_en'] , 


    # fig2.add_hline(y = line_20, line_width=3, line_dash="dash", line_color="black", opacity = .4,\
    #     annotation_text="2020 <br> weighted mean profit_rate <br>, %s" %line_20 , 
    #     annotation_position="top right", annotation_opacity = .4, col = 1)
    # fig2.add_hline(y = line_21, line_width=3, line_dash="dash", line_color="black", opacity = .4,\
    #     annotation_text="2021 <br> weighted mean profit_rate <br>, %s" %line_21 , 
    #     annotation_position="top right", annotation_opacity = .4, col = 2)

    return fig2

plot_('2021-11-28')